<a href="https://colab.research.google.com/github/irfqn/MachineLearningTasks/blob/main/13WeekTasks/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Impor library yang diperlukan
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Tentukan perangkat ke GPU jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tentukan arsitektur LeNet-5 untuk MNIST
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # Lapisan konvolusional pertama
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Lapisan konvolusional kedua
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Lapisan fully connected
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.relu3 = nn.ReLU()

        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()

        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Langkah maju melalui jaringan
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.relu3(self.fc1(x))
        x = self.relu4(self.fc2(x))
        x = self.fc3(x)
        return x

# Muat dataset MNIST
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Inisialisasi model, fungsi kerugian, dan pengoptimal
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Pelatihan
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Nolkan gradien
        optimizer.zero_grad()

        # Langkah maju
        outputs = model(inputs)
        # Hitung kerugian
        loss = criterion(outputs, labels)
        # Langkah mundur dan optimisasi
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Cetak rata-rata kerugian untuk epoch tersebut
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

# Pengujian
model.eval()
total_correct = 0
total_samples = 0

# Evaluasi model pada set pengujian
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Langkah maju
        outputs = model(inputs)
        # Dapatkan prediksi
        _, predictions = torch.max(outputs, 1)

        # Perbarui total prediksi benar dan total sampel
        total_correct += (predictions == labels).sum().item()
        total_samples += labels.size(0)

# Hitung dan cetak akurasi pengujian
accuracy = total_correct / total_samples
print(f"Akurasi Pengujian: {accuracy}")

100%|██████████| 9912422/9912422 [00:00<00:00, 104066238.92it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 19360027.78it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 24756197.73it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3140024.52it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Epoch 1/5, Loss: 0.2755207035517785
Epoch 2/5, Loss: 0.07857905867308981
